In [24]:
import streamlit as st
from pathlib import Path
from langchain.agents import create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from sqlalchemy import create_engine
import sqlite3
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
## load the GROQ API Key
os.environ["groq_API"]=os.getenv("groq_API")
groq_api_key=os.getenv("groq_API")

In [25]:
st.set_page_config(page_title="LangChain: Chat with SQL DB", page_icon=r"C:\Users\99ash\Downloads\mysql-create-database-workbench.png")
st.image(r"C:\Users\99ash\Downloads\mysql-create-database-workbench.png")
st.title(" LangChain: Chat with SQL DB")

LOCALDB="USE_LOCALDB"
MYSQL="USE_MYSQL"

radio_opt=["Use SQLLite 3 Database- Student.db","Connect to you MySQL Database"]

selected_opt=st.sidebar.radio(label="Choose the DB which you want to chat",options=radio_opt)

2025-01-29 18:13:15.312 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.314 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.316 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.317 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.318 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.319 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.320 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.321 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [26]:
if radio_opt.index(selected_opt)==1:
    db_uri=MYSQL
    mysql_host=st.sidebar.text_input("Provide MySQL Host")
    mysql_user=st.sidebar.text_input("MYSQL User")
    mysql_password=st.sidebar.text_input("MYSQL password",type="password")
    mysql_db=st.sidebar.text_input("MySQL database")
else:
    db_uri=LOCALDB

In [27]:
api_key=st.sidebar.text_input(label="GRoq API Key",type="password")
if not db_uri:
    st.info("Please enter the database information and uri")

if not api_key:
    st.info("Please add the groq api key")
## LLM model
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-70b-8192",streaming=True)

2025-01-29 18:13:15.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.361 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [28]:
@st.cache_resource(ttl="2h")
def configure_db(db_uri,mysql_host=None,mysql_user=None,mysql_password=None,mysql_db=None):
    if db_uri==LOCALDB:
        dbfilepath=r"c:\Users\99ash\student.db"
        creator = lambda: sqlite3.connect(f"file:{dbfilepath}?mode=ro", uri=True)
        return SQLDatabase(create_engine("sqlite:///", creator=creator))
    elif db_uri==MYSQL:
        if not (mysql_host and mysql_user and mysql_password and mysql_db):
            st.error("Please provide all MySQL connection details.")
            st.stop()
        return SQLDatabase(create_engine(f"mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_db}"))   
    
if db_uri==MYSQL:
    db=configure_db(db_uri,mysql_host,mysql_user,mysql_password,mysql_db)
else:
    db=configure_db(db_uri)

In [29]:
## toolkit
toolkit=SQLDatabaseToolkit(db=db,llm=llm)

agent=create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [30]:
if "messages" not in st.session_state or st.sidebar.button("Clear message history"):
    st.session_state["messages"] = [{"role": "assistant", "content": "How can I help you?"}]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

user_query=st.chat_input(placeholder="Ask anything from the database")

if user_query:
    st.session_state.messages.append({"role": "user", "content": user_query})
    st.chat_message("user").write(user_query)

    with st.chat_message("assistant"):
        streamlit_callback=StreamlitCallbackHandler(st.container())
        response=agent.run(user_query,callbacks=[streamlit_callback])
        st.session_state.messages.append({"role":"assistant","content":response})
        st.write(response)

2025-01-29 18:13:15.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.510 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.511 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.513 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-29 18:13:15.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar